In [1]:
%store -r concat_df
%store -r data_avg
%store -r res

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import frameprocessing

In [2]:

data_avg = data_avg.reset_index()
concat_df

#concat_df.loc[concat_df['Date'] == "2022-05-29"]
concat_df["unique string"] = concat_df.apply(lambda x: str(x["Date"]) + str(x["Home_team"]),axis=1)
concat_df = concat_df.drop_duplicates(subset = ["unique string"]).drop(["unique string"],axis=1)



#best_df = pd.DataFrame([])
#best_df.columns = concat_df.columns
#for row in concat_df:
#    if row["unique_string"] in 



In [3]:
concat_df.loc[concat_df['Date'] == "2021-12-04"]


hot_data_avg = pd.get_dummies(data_avg["Team"])

avg_data_joined = data_avg.join(hot_data_avg)

data_avg_opponent = data_avg.copy(deep=True)
data_avg_opponent["Team_jord"] = frameprocessing.getCurrentTeamRows(data_avg_opponent)
hot_data_avg_opponent = pd.get_dummies(data_avg_opponent["Team_jord"])

avg_data_joined_opponent = data_avg_opponent.join(hot_data_avg_opponent)
avg_data_joined_opponent.drop(["Team_jord"],axis=1,inplace=True)
avg_data_joined_opponent

In [4]:
def getCurrentTeamRows(df):
    temp = ['current_team '] * len(df['Team'])
    names = df['Team']

    current_columns = [y+x for x,y in zip(names,temp)]
    return current_columns





In [5]:
def x_transform(df, avg_df):
    None


In [6]:
concat_df = concat_df.loc[concat_df['Year']=="2021"]
concat_df.drop(["Year","Result"],axis=1,inplace=True)
concat_df = frameprocessing.one_hot_etc(concat_df)
X = concat_df.drop(['result_label'], axis=1)

y = concat_df['result_label']
#X = np.asarray(X).astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#vi vill att X_test ska ha hemmalag och datum
X_avg = frameprocessing.x_test_transform(X_test['Team'],X_test['opponent Team'],avg_df = avg_data_joined,opp_avg_df = avg_data_joined_opponent,home = X_test)
#X_test
#X_avg
#X_avg= frameprocessing.one_hot_etc(X_avg)
#X_test
X_test.drop(["opponent Team","Team"],axis=1,inplace=True)
X_avg.drop(["opponent Team","Team"],axis=1,inplace=True)
X_train.drop(["opponent Team","Team"],axis=1,inplace=True)
#concat_df


['current_team Hammarby', 'current_team Varberg', 'current_team Degerfors', 'current_team IFK Göteborg', 'current_team Hammarby', 'current_team Sirius', 'current_team Örebro', 'current_team AIK', 'current_team Örebro', 'current_team Halmstad', 'current_team Elfsborg', 'current_team Degerfors', 'current_team Halmstad', 'current_team IFK Norrköping', 'current_team Halmstad', 'current_team Varberg', 'current_team AIK', 'current_team Halmstad', 'current_team Östersunds FK', 'current_team IFK Göteborg', 'current_team Elfsborg', 'current_team AIK', 'current_team Djurgården', 'current_team Hammarby', 'current_team Halmstad', 'current_team Elfsborg', 'current_team Östersunds FK', 'current_team Halmstad', 'current_team Varberg', 'current_team Djurgården', 'current_team Degerfors', 'current_team Elfsborg', 'current_team Hammarby', 'current_team Sirius', 'current_team Varberg', 'current_team AIK', 'current_team Elfsborg', 'current_team IFK Norrköping', 'current_team Degerfors', 'current_team Hamm

/Users/oskarasbrink/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
X_testmannen = X_test[['Date','Home TEAM NAME']]
#X_train = frameprocessing.one_hot_etc(X_train)
#X_test = frameprocessing.one_hot_etc(X_test)
#X_test2 = X_test.copy(deep=True)

In [8]:
encoder = LabelEncoder()

encoder.fit(y)
encoded_y_tjark = encoder.transform(y)
dummy = np_utils.to_categorical(encoded_y_tjark)

encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
dummy_y = np_utils.to_categorical(encoded_Y)

encoder.fit(y_test)
encoded_Y_test = encoder.transform(y_test)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)


X_train = X_train.drop(['Home TEAM NAME',"Home_team","opponent Home_team","opponent Result","Date"],axis=1)
X_test = X_test.drop(['Home TEAM NAME',"Home_team","opponent Home_team","opponent Result","Date"],axis=1)
pd.set_option("display.max_columns", None)
#X_avg = X_avg.drop(['Home TEAM NAME',"Home_team","opponent Home_team","opponent Result","Date"],axis=1)



In [9]:
X_train

,Goals,xG,Shots / on target,Unnamed: 9,Unnamed: 10,Passes / accurate,Unnamed: 12,Unnamed: 13,"Possession, %",Losses / Low / Medium / High,Unnamed: 16,Unnamed: 17,Unnamed: 18,Recoveries / Low / Medium / High,Unnamed: 20,Unnamed: 21,Unnamed: 22,Duels / won,Unnamed: 24,Unnamed: 25,Shots from outside penalty area / on target,Unnamed: 27,Unnamed: 28,Positional attacks / with shots,Unnamed: 30,Unnamed: 31,Counterattacks / with shots,Unnamed: 33,Unnamed: 34,Set pieces / with shots,Unnamed: 36,Unnamed: 37,Corners / with shots,Unnamed: 39,Unnamed: 40,Free kicks / with shots,Unnamed: 42,Unnamed: 43,Penalties / converted,Unnamed: 45,Unnamed: 46,Crosses / accurate,Unnamed: 48,Unnamed: 49,Deep completed crosses,Deep completed passes,Penalty area entries (runs / crosses),Unnamed: 53,Unnamed: 54,Touches in penalty area,Offensive duels / won,Unnamed: 57,Unnamed: 58,Offsides,Conceded goals,Shots against / on target,Unnamed: 62,Unnamed: 63,Defensive duels / won,Unnamed: 65,Unnamed: 66,Aerial duels / won,Unnamed: 68,Unnamed: 69,Sliding tackles / successful,Unnamed: 71,Unnamed: 72,Interceptions,Clearances,Fouls,Yellow cards,Red cards,Forward passes / accurate,Unnamed: 79,Unnamed: 80,Back passes / accurate,Unnamed: 82,Unnamed: 83,Lateral passes / accurate,Unnamed: 85,Unnamed: 86,Long passes / accurate,Unnamed: 88,Unnamed: 89,Passes to final third / accurate,Unnamed: 91,Unnamed: 92,Progressive passes / accurate,Unnamed: 94,Unnamed: 95,Smart passes / accurate,Unnamed: 97,Unnamed: 98,Throw ins / accurate,Unnamed: 100,Unnamed: 101,Goal kicks,Match tempo,Average passes per possession,Long pass %,Average shot distance,Average pass length,PPDA,Home,opponent Goals,opponent xG,opponent Shots / on target,opponent Unnamed: 9,opponent Unnamed: 10,opponent Passes / accurate,opponent Unnamed: 12,opponent Unnamed: 13,"opponent Possession, %",opponent Losses / Low / Medium / High,opponent Unnamed: 16,opponent Unnamed: 17,opponent Unnamed: 18,opponent Recoveries / Low / Medium / High,opponent Unnamed: 20,opponent Unnamed: 21,opponent Unnamed: 22,opponent Duels / won,opponent Unnamed: 24,opponent Unnamed: 25,opponent Shots from outside penalty area / on target,opponent Unnamed: 27,opponent Unnamed: 28,opponent Positional attacks / with shots,opponent Unnamed: 30,opponent Unnamed: 31,opponent Counterattacks / with shots,opponent Unnamed: 33,opponent Unnamed: 34,opponent Set pieces / with shots,opponent Unnamed: 36,opponent Unnamed: 37,opponent Corners / with shots,opponent Unnamed: 39,opponent Unnamed: 40,opponent Free kicks / with shots,opponent Unnamed: 42,opponent Unnamed: 43,opponent Penalties / converted,opponent Unnamed: 45,opponent Unnamed: 46,opponent Crosses / accurate,opponent Unnamed: 48,opponent Unnamed: 49,opponent Deep completed crosses,opponent Deep completed passes,opponent Penalty area entries (runs / crosses),opponent Unnamed: 53,opponent Unnamed: 54,opponent Touches in penalty area,opponent Offensive duels / won,opponent Unnamed: 57,opponent Unnamed: 58,opponent Offsides,opponent Conceded goals,opponent Shots against / on target,opponent Unnamed: 62,opponent Unnamed: 63,opponent Defensive duels / won,opponent Unnamed: 65,opponent Unnamed: 66,opponent Aerial duels / won,opponent Unnamed: 68,opponent Unnamed: 69,opponent Sliding tackles / successful,opponent Unnamed: 71,opponent Unnamed: 72,opponent Interceptions,opponent Clearances,opponent Fouls,opponent Yellow cards,opponent Red cards,opponent Forward passes / accurate,opponent Unnamed: 79,opponent Unnamed: 80,opponent Back passes / accurate,opponent Unnamed: 82,opponent Unnamed: 83,opponent Lateral passes / accurate,opponent Unnamed: 85,opponent Unnamed: 86,opponent Long passes / accurate,opponent Unnamed: 88,opponent Unnamed: 89,opponent Passes to final third / accurate,opponent Unnamed: 91,opponent Unnamed: 92,opponent Progressive passes / accurate,opponent Unnamed: 94,opponent Unnamed: 95,opponent Smart passes / accurate,opponent Unnamed: 97,opponent Unnamed: 98,opponent Throw ins / accurate,

In [10]:
X_avg

,index,Goals,xG,Shots / on target,Unnamed: 9,Unnamed: 10,Passes / accurate,Unnamed: 12,Unnamed: 13,"Possession, %",Losses / Low / Medium / High,Unnamed: 16,Unnamed: 17,Unnamed: 18,Recoveries / Low / Medium / High,Unnamed: 20,Unnamed: 21,Unnamed: 22,Duels / won,Unnamed: 24,Unnamed: 25,Shots from outside penalty area / on target,Unnamed: 27,Unnamed: 28,Positional attacks / with shots,Unnamed: 30,Unnamed: 31,Counterattacks / with shots,Unnamed: 33,Unnamed: 34,Set pieces / with shots,Unnamed: 36,Unnamed: 37,Corners / with shots,Unnamed: 39,Unnamed: 40,Free kicks / with shots,Unnamed: 42,Unnamed: 43,Penalties / converted,Unnamed: 45,Unnamed: 46,Crosses / accurate,Unnamed: 48,Unnamed: 49,Deep completed crosses,Deep completed passes,Penalty area entries (runs / crosses),Unnamed: 53,Unnamed: 54,Touches in penalty area,Offensive duels / won,Unnamed: 57,Unnamed: 58,Offsides,Conceded goals,Shots against / on target,Unnamed: 62,Unnamed: 63,Defensive duels / won,Unnamed: 65,Unnamed: 66,Aerial duels / won,Unnamed: 68,Unnamed: 69,Sliding tackles / successful,Unnamed: 71,Unnamed: 72,Interceptions,Clearances,Fouls,Yellow cards,Red cards,Forward passes / accurate,Unnamed: 79,Unnamed: 80,Back passes / accurate,Unnamed: 82,Unnamed: 83,Lateral passes / accurate,Unnamed: 85,Unnamed: 86,Long passes / accurate,Unnamed: 88,Unnamed: 89,Passes to final third / accurate,Unnamed: 91,Unnamed: 92,Progressive passes / accurate,Unnamed: 94,Unnamed: 95,Smart passes / accurate,Unnamed: 97,Unnamed: 98,Throw ins / accurate,Unnamed: 100,Unnamed: 101,Goal kicks,Match tempo,Average passes per possession,Long pass %,Average shot distance,Average pass length,PPDA,AIK,Degerfors,Djurgården,Elfsborg,GIF Sundsvall,Hammarby,Helsingborg,Häcken,IFK Göteborg,IFK Norrköping,Kalmar,Malmö FF,Mjällby,Sirius,Varberg,Värnamo,Home,index,opponent Goals,opponent xG,opponent Shots / on target,opponent Unnamed: 9,opponent Unnamed: 10,opponent Passes / accurate,opponent Unnamed: 12,opponent Unnamed: 13,"opponent Possession, %",opponent Losses / Low / Medium / High,opponent Unnamed: 16,opponent Unnamed: 17,opponent Unnamed: 18,opponent Recoveries / Low / Medium / High,opponent Unnamed: 20,opponent Unnamed: 21,opponent Unnamed: 22,opponent Duels / won,opponent Unnamed: 24,opponent Unnamed: 25,opponent Shots from outside penalty area / on target,opponent Unnamed: 27,opponent Unnamed: 28,opponent Positional attacks / with shots,opponent Unnamed: 30,opponent Unnamed: 31,opponent Counterattacks / with shots,opponent Unnamed: 33,opponent Unnamed: 34,opponent Set pieces / with shots,opponent Unnamed: 36,opponent Unnamed: 37,opponent Corners / with shots,opponent Unnamed: 39,opponent Unnamed: 40,opponent Free kicks / with shots,opponent Unnamed: 42,opponent Unnamed: 43,opponent Penalties / converted,opponent Unnamed: 45,opponent Unnamed: 46,opponent Crosses / accurate,opponent Unnamed: 48,opponent Unnamed: 49,opponent Deep completed crosses,opponent Deep completed passes,opponent Penalty area entries (runs / crosses),opponent Unnamed: 53,opponent Unnamed: 54,opponent Touches in penalty area,opponent Offensive duels / won,opponent Unnamed: 57,opponent Unnamed: 58,opponent Offsides,opponent Conceded goals,opponent Shots against / on target,opponent Unnamed: 62,opponent Unnamed: 63,opponent Defensive duels / won,opponent Unnamed: 65,opponent Unnamed: 66,opponent Aerial duels / won,opponent Unnamed: 68,opponent Unnamed: 69,opponent Sliding tackles / successful,opponent Unnamed: 71,opponent Unnamed: 72,opponent Interceptions,opponent Clearances,opponent Fouls,opponent Yellow cards,opponent Red cards,opponent Forward passes / accurate,opponent Unnamed: 79,opponent Unnamed: 80,opponent Back passes / accurate,opponent Unnamed: 82,opponent Unnamed: 83,opponent Lateral passes / accurate,opponent Unnamed: 85,opponent Unnamed: 86,opponent Long passes / accurate,opponent Unnamed: 88,opponent Unnamed: 89,opponent Passes to final third / accurate,opponent Unnamed: 91,opponent Unnamed: 92,opponent Progressive pa

In [11]:
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
X_avg = np.asarray(X_avg).astype('float32')
model = Sequential()
model.add(Dense(len(X_train[0]), input_dim = len(X_train[0]), activation='relu'))
#model.add(Dense(64, input_dim=len(X_test.drop(['index'],axis = 1).columns), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, dummy_y, epochs=100, batch_size=100)

2022-07-16 21:45:20.426595: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-16 21:45:20.426849: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-16 21:45:20.507177: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
2/2 [==============================] - 0s 3ms/step - loss: 90.9746 - accuracy: 0.3157
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 28.2768 - accuracy: 0.4324
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 34.6160 - accuracy: 0.4395
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 28.7340 - accuracy: 0.4505
Epoch 5/100
2/2 [==============================] - 0s 2ms/step - loss: 13.4862 - accuracy: 0.4219
Epoch 6/100
2/2 [==============================] - 0s 3ms/step - loss: 9.7714 - accuracy: 0.4262
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 12.0935 - accuracy: 0.4300
Epoch 8/100
2/2 [==============================] - 0s 3ms/step - loss: 9.5155 - accuracy: 0.4643
Epoch 9/100
2/2 [==============================] - 0s 3ms/step - loss: 7.5383 - accuracy: 0.3305
Epoch 10/100
2/2 [==============================] - 0s 2ms/step - loss: 3.8782 - accuracy: 0.5276
Epoch 11/100
2/2 [=====

In [12]:
#y_pred = model.predict(X_test2.drop(['index'],axis=1))

y_pred = model.predict(X_avg)#2.drop(['index'],axis=1))
print(np.around(y_pred, 2))
#-train_test_split
#-model predict
#-jämför med true värden från res


ValueError: in user code:

    /Users/oskarasbrink/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /Users/oskarasbrink/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/oskarasbrink/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/oskarasbrink/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/oskarasbrink/.pyenv/versions/3.7.0/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
    

In [ ]:
X_testmannen["Date"] =  X_testmannen['Date'].apply(lambda x: x.replace("-","/"))
X_testmannen["Date"] = X_testmannen["Date"].apply(lambda x: frameprocessing.dateSwitcher(x))
print(X_testmannen.shape[0])
#X_testmannen
X_testmannen["Home TEAM NAME"] = X_testmannen["Home TEAM NAME"].apply(lambda x: frameprocessing.transformTeam(x))
test_df = X_testmannen.merge(res, how = "inner",left_on = ['Date',"Home TEAM NAME"],right_on=['Date','Home_team'])
test_df


y_true = test_df[["AvgH","AvgD","AvgA"]]
y_true = np.asarray(y_true).astype('float32')




In [ ]:
y_pred